# Model Evaluation With WER

Kelompok 17 - Mebious: \
Hendrik Nicolas Carlo – 2602124171 \
Kelvin Andreas – 2602121794 \
Kevin Petersen – 2602136713


## Installing library

In [1]:
!pip install datasets jiwer evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 11.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompat

## Loading dataset mozilla common voices 17

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset, DatasetDict
from google.colab import userdata

common_voice = DatasetDict()
common_voice = load_dataset("mozilla-foundation/common_voice_17_0", "id", split="validation", token=userdata.get('HF_write'))
print(common_voice)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for mozilla-foundation/common_voice_17_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_17_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 4970it [00:00, 74151.76it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 3349it [00:00, 70794.67it/s]


...

Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 2605it [00:00, 87104.08it/s]


Generating validated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 10165it [00:00, 101642.83it/s]
Reading metadata...: 26108it [00:00, 113584.74it/s]


Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
    num_rows: 3349
})


In [4]:
common_voice = common_voice.remove_columns(["accent", "variant", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])
common_voice = common_voice.select(range(100))

## Transcribing the Test data with whisper models
how we transcibe

### Whisper Tiny

In [5]:
from transformers import pipeline
pipe = pipeline(model="openai/whisper-tiny")

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [6]:
transcript_tiny = []
truth = []
for i in range(100):
  transcript_tiny.append(transcribe(common_voice[i]['audio']))
  truth.append(common_voice[i]['sentence'])

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


In [7]:
from jiwer import wer

error_tiny = wer(reference=truth, hypothesis=transcript_tiny)

### Whisper Tiny Fine Tuned

In [8]:
pipe = pipeline(model="ChronoStellar/whisper-tiny-id")

transcript_tiny_tuned = []
for i in range(100):
  transcript_tiny_tuned.append(transcribe(common_voice[i]['audio']))

error_tiny_tuned = wer(reference=truth, hypothesis=transcript_tiny_tuned)

config.json:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.70k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

### Whisper Base

In [9]:
pipe = pipeline(model="openai/whisper-base")

transcript_base = []
for i in range(100):
  transcript_base.append(transcribe(common_voice[i]['audio']))

error_base = wer(reference=truth, hypothesis=transcript_base)

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.81k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

### Whisper Small

In [10]:
pipe = pipeline(model="openai/whisper-small")

transcript_small = []
for i in range(100):
  transcript_small.append(transcribe(common_voice[i]['audio']))

error_small = wer(reference=truth, hypothesis=transcript_small)

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

### transcript result

In [12]:
import pandas as pd
df = pd.DataFrame({
    'transcript_tiny': transcript_tiny,
    'transcript_tiny_tuned': transcript_tiny_tuned,
    'transcript_base': transcript_base,
    'transcript_small': transcript_small
})

df

,transcript_tiny,transcript_tiny_tuned,transcript_base,transcript_small
0,Kempa bisa terjadi kapan saat ken.,Kempa bisa terjadi kapan sakin.,Kempa bisa terjadi kapan sadian.,Gepa bisa terjadi kapan saja.
1,selasatu pantai di pulau beritung dan aku kur...,Salah satu pantai di pulau blito yang aku kunj...,salah satu pantai di pulau blitum yang ngeaku...,salah satu pantai di pulau beli itu yang aku ...
2,Исана Банякто по бреде терр.,Di sana banyak topo berjajer.,"Бесона, баняк топового бердетия.",disana banyak toko berjajir
3,setelah pulang sekolah setiap hari saya berba...,"Setelah pulang sekolah, setiap hari saya berba...",setelah kulang sekolah setiap hari saya berla...,Setelah pulang sekolah setiap hari saya berla...
4,Kamu bisa behoong pada orang lain tetapi kamu...,"Kamu bisa behoem pada orang lain, tetapi kamu ...","Kamu bisa bohong pada orang lain, tetapi kamu...","Kamu bisa bohong pada orang lain, tetapi kamu..."
...,...,...,...,...
95,"Tira, ¿vos eres más su familia?",Tidak boleh masuk ke marine.,di daubolah masuk kamar ini,tidak boleh masuk kamar ini
96,"Sejauh yang saya tahu, tom tidak pernah menco...","Sejauh yang saya tahu, Tom tidak pernah mencob...","Sejauh yang saya tahu, tonton tidak pernah me...","Sejauh yang saya tahu, Tom tidak pernah menco..."
97,dia berharap dia memiliki lebih banyak waktu,Dia berharap dia memiliki lebih banyak waktu.,"Dia berharap dia maksudnya, aku laki lebih ba...",Dia berharap dia memiliki lebih banyak waktu.
98,Monellisah mungkin ada lokisahan paling terke...,Monalisa mungkin adalah lokisan paling terkena...,Mona Lisa mungkin ada lo kissan paling terken...,Mona Lisa mungkin ada lokisan paling terkenal...


## Evaluating the WER

In [22]:
print(f'whisper tiny WER \t\t: {round(error_tiny, 4)*100}%, 2 minutes')
print(f'whisper tiny tuned WER \t\t: {round(error_tiny_tuned, 4)*100}%, 3 minutes')
print(f'whisper base WER \t\t: {round(error_base, 4)*100}%, 4 minutes')
print(f'whisper small WER \t\t: {round(error_small, 4)*100}%, 15 minutes')

eval = pd.DataFrame({
    'transcript_tiny': [error_tiny],
    'transcript_tiny_tuned': [error_tiny_tuned],
    'transcript_base': [error_base],
    'transcript_small': [error_small]
})

whisper tiny WER 		: 63.4%, 2 minutes
whisper tiny tuned WER 		: 27.29%, 3 minutes
whisper base WER 		: 40.93%, 4 minutes
whisper small WER 		: 20.06%, 15 minutes


In [23]:
eval

,transcript_tiny,transcript_tiny_tuned,transcript_base,transcript_small
0,0.634029,0.272873,0.40931,0.200642
